In [1]:
!pip3 install pandasql

  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [0]:
import numpy as np
import pandas as pd
import pandasql as ps
from time import time

In [29]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data'
data_diagnostic = pd.read_csv(url, sep=',',header=None)
data_diagnostic.columns = ['id'] + ['D' + str(i) for i in range(1, 32)]
data_diagnostic.head()

,id,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D22,D23,D24,D25,D26,D27,D28,D29,D30,D31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [30]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wpbc.data'
data_prognostic = pd.read_csv(url, sep=',',header=None)
data_prognostic.columns = ['id'] + ['P' + str(i) for i in range(1, 35)]
data_prognostic.head()

,id,P1,P2,P3,P4,P5,P6,P7,P8,P9,...,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34
0,119513,N,31,18.02,27.60,117.50,1013.0,0.09489,0.1036,0.1086,...,139.70,1436.0,0.1195,0.1926,0.3140,0.1170,0.2677,0.08113,5.0,5
1,8423,N,61,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.3001,...,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,3.0,2
2,842517,N,116,21.37,17.44,137.50,1373.0,0.08836,0.1189,0.1255,...,159.10,1949.0,0.1188,0.3449,0.3414,0.2032,0.4334,0.09067,2.5,0
3,843483,N,123,11.42,20.38,77.58,386.1,0.14250,0.2839,0.2414,...,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,2.0,0
4,843584,R,27,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.1980,...,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,0


In [31]:
# 1 - diagnosis, 2 - radius, 22 - worst radius
t0_diagnostic = time()
aggregations_diagnostic = {
    'D22': lambda x: max(x)    
}
print(data_diagnostic.groupby('D1').agg(aggregations_diagnostic))
t1_diagnostic = time()
print('It takes: ', t1_diagnostic - t0_diagnostic)

      D22
D1       
B   19.82
M   36.04
It takes:  0.005372047424316406


In [23]:
# 1 - diagnosis, 2 - radius, 22 - worst radius
t0_diagnostic = time()
aggr_query_diagnostic = '''
    SELECT 
        D1,
        max(D22)
    FROM data_diagnostic 
    GROUP BY D1
    '''
print(ps.sqldf(aggr_query_diagnostic, locals()))
t1_diagnostic = time()
print('It takes: ', t1_diagnostic - t0_diagnostic)

  D1  max(D22)
0  B     19.82
1  M     36.04
It (diagnostic) takes:  0.04991793632507324


In [37]:
t0 = time()
result = pd.merge(data_diagnostic[['id','D1', 'D2', 'D22']],
                 data_prognostic[['id', 'P1', 'P2']],
                 on='id')
t1 = time()
print(result)
print('It takes: ', t1 - t0)

          id D1     D2    D22 P1   P2
0     842517  M  20.57  24.99  N  116
1     843786  M  12.45  15.47  R   77
2     844359  M  18.25  22.88  N   60
3     844981  M  13.00  15.49  N  119
4     845636  M  16.02  19.19  N  123
5     846381  M  15.85  16.84  N  117
6     848406  M  14.68  19.07  N  123
7    8511133  M  15.34  18.07  N   20
8     851509  M  21.16  29.17  R   10
9     852552  M  16.65  26.46  N   96
10    852631  M  17.14  22.25  N  116
11    852763  M  14.58  17.62  N  103
12    852781  M  18.61  21.31  N   16
13    852973  M  15.30  20.27  N   52
14    853201  M  17.57  20.01  N   94
15    853612  M  11.84  16.82  N  116
16    854002  M  19.27  24.15  N   53
17    854039  M  16.13  20.21  N  109
18    854253  M  16.74  20.01  N   12
19    854268  M  14.25  15.89  N   31
20    855133  M  14.99  14.99  N   72
21    855138  M  13.48  15.53  N   76
22    855167  M  13.44  15.93  N   34
23    855563  M  10.95  12.84  N   99
24    855625  M  19.07  24.09  R    9
25    856106

In [36]:
t0 = time()
merge_query = '''
    SELECT 
        d.id,
        d.D1,
        d.D2,
        d.D22,
        p.id, 
        p.P1,
        p.P2
    FROM data_diagnostic as d JOIN data_prognostic as p ON (d.id = p.id)
    GROUP BY p.id
    '''
print(ps.sqldf(merge_query, locals()))
t1 = time()
print('It takes: ', t1 - t0)

          id D1     D2    D22       id P1   P2
0      85715  M  13.17  15.67    85715  N  111
1      86208  M  20.26  24.22    86208  R   10
2      86517  M  18.66  22.25    86517  N  108
3      87163  M  13.43  17.98    87163  N   84
4      87164  M  15.46  18.79    87164  N   98
5      87880  M  13.81  19.20    87880  N   38
6      89122  M  19.40  23.79    89122  N   17
7      89812  M  23.51  30.67    89812  R   24
8      90291  M  14.60  15.79    90291  N   57
9      90312  M  19.55  20.82    90312  N   13
10     91485  M  20.59  23.86    91485  R   11
11     91504  M  13.82  16.01    91504  N   34
12    842517  M  20.57  24.99   842517  N  116
13    843786  M  12.45  15.47   843786  R   77
14    844359  M  18.25  22.88   844359  N   60
15    844981  M  13.00  15.49   844981  N  119
16    845636  M  16.02  19.19   845636  N  123
17    846381  M  15.85  16.84   846381  N  117
18    848406  M  14.68  19.07   848406  N  123
19    851509  M  21.16  29.17   851509  R   10
20    852552 